# MNIST convolutional neural network (Deep MNIST for Experts)
----------------------
[Tutorial link](https://www.tensorflow.org/get_started/mnist/pros)

Dependencies
- Python 3.6
- Tensorflow 1.2
- tqdm

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm_notebook as tqdm

## Build a Multilayer Convolutional Network
--------------------------------------
1. reshape MNIST input data
2. layer1: 5x5 convolution and pooling
3. layer2: 3x3 convolution and pooling
4. flatten (reshape) activation map
5. layer3: 1024 fully connected layer
6. output layer: 10 fully connected classification layer

We will use 'tf.reshape', 'tf.nn.conv2d', 'tf.relu', 'tf.dropout', and 'tf.nn.softmax_cross_entropy_with_logits'

In [2]:
# 0. set placeholder for input data and dropout probability
x = tf.placeholder(tf.float32, [None, 784])
y_label = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

In [3]:
# 1. reshape MNIST input data
x_image = tf.reshape(x, [-1, 28, 28, 1]) # 'NHWC' format (batch_size, height, width, channel)

In [4]:
# 2. layer1: 5x5 convolution and pooling
W_conv1 = tf.Variable(tf.random_normal([5, 5, 1, 32], stddev=0.2), # convolution with 5x5 filter, 32 features
                      name='conv1_weight', dtype=tf.float32)
b_conv1 = tf.Variable(tf.zeros([32]), name='conv1_bias')
conv1 = tf.nn.conv2d(x_image, W_conv1, [1,1,1,1], padding='SAME')
conv1 = tf.nn.bias_add(conv1, b_conv1)
conv1 = tf.nn.relu(conv1) # add activation function
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
pool1 = tf.nn.dropout(pool1, dropout_prob)

In [5]:
# 3. layer2: 3x3 convolution and pooling
W_conv2 = tf.Variable(tf.random_normal([3, 3, pool1.get_shape().as_list()[-1], 64], stddev=0.2),
                      name='conv2_weight', dtype=tf.float32)
b_conv2 = tf.Variable(tf.zeros([64]), name='conv2_bias')
conv2 = tf.nn.conv2d(pool1, W_conv2, [1,1,1,1], padding='SAME')
conv2 = tf.nn.bias_add(conv2, b_conv2)
conv2 = tf.nn.relu(conv2)
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
pool2 = tf.nn.dropout(pool2, dropout_prob)

In [6]:
# 4. flatten (reshape) activation map
total_dim = 1
for dim in pool2.get_shape().as_list()[1:]:
    total_dim *= dim
print(total_dim)
    
pool2_flat = tf.reshape(pool2, [-1, total_dim])

3136


In [7]:
# 5. layer3: 1024 fully connected layer
W_fc1 = tf.Variable(tf.random_uniform([total_dim, 1024], -0.03, 0.03),
                    name='fc1_weight', dtype=tf.float32)
b_fc1 = tf.Variable(tf.zeros([1024]),  name='fc1_bias')
fc1 = tf.matmul(pool2_flat, W_fc1) + b_fc1
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, dropout_prob)

In [8]:
# 6. output layer: 10 fully connected classification layer
W_fc2 = tf.Variable(tf.random_uniform([fc1.get_shape().as_list()[-1], 10], -0.08, 0.08),
                   name='fc2_weight', dtype=tf.float32)
b_fc2 = tf.Variable(tf.zeros([10]))
fc2 = tf.matmul(fc1, W_fc2) + b_fc2

In [9]:
print(conv1)
print(pool1)
print(conv2)
print(pool2)
print(fc1)
print(fc2)

Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("dropout_1/mul:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 1024), dtype=float32)
Tensor("add_1:0", shape=(?, 10), dtype=float32)


## Set loss and optimizer

In [10]:
# logits needs unscaled log probability.
# It means we need to feed output value itself. 
# So, we do not apply activation function to 'fc2'.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=fc2))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

## Create prediction op

In [11]:
prediction = tf.equal(tf.argmax(fc2,1), tf.argmax(y_label,1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

## Run session

In [12]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
for i in tqdm(range(20000)):
    images, labels = mnist.train.next_batch(128)
    if i % 1000 == 0:
        pred, acc = sess.run([prediction, accuracy], feed_dict={x: images, y_label: labels, dropout_prob: 1.0})
        print("step {: 5d}, train accuracy[{: 2d}/{: 3d}] {:.5f}"
              .format(i, pred.sum(), pred.shape[0], acc))
    optimizer.run(feed_dict={x: images, y_label: labels, dropout_prob: 0.5})

step     0, train accuracy[ 11/ 128] 0.08594
step  1000, train accuracy[ 119/ 128] 0.92969
step  2000, train accuracy[ 121/ 128] 0.94531
step  3000, train accuracy[ 128/ 128] 1.00000
step  4000, train accuracy[ 123/ 128] 0.96094
step  5000, train accuracy[ 125/ 128] 0.97656
step  6000, train accuracy[ 128/ 128] 1.00000
step  7000, train accuracy[ 126/ 128] 0.98438
step  8000, train accuracy[ 127/ 128] 0.99219
step  9000, train accuracy[ 127/ 128] 0.99219
step  10000, train accuracy[ 128/ 128] 1.00000
step  11000, train accuracy[ 127/ 128] 0.99219
step  12000, train accuracy[ 128/ 128] 1.00000
step  13000, train accuracy[ 128/ 128] 1.00000
step  14000, train accuracy[ 127/ 128] 0.99219
step  15000, train accuracy[ 127/ 128] 0.99219
step  16000, train accuracy[ 127/ 128] 0.99219
step  17000, train accuracy[ 127/ 128] 0.99219
step  18000, train accuracy[ 128/ 128] 1.00000
step  19000, train accuracy[ 128/ 128] 1.00000



In [14]:
pred, acc = sess.run([prediction, accuracy],
                    feed_dict={x: mnist.test.images, 
                                y_label: mnist.test.labels,
                                dropout_prob: 1.0})
print("test accuracy[{: 5d}/{: 5d}] {:.5f}"
      .format(pred.sum(), pred.shape[0], acc))

ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,32]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape, conv1_weight/read)]]
	 [[Node: Equal/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_108_Equal", tensor_type=DT_BOOL, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D', defined at:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/grayish/PyVirtualEnv/py36-tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/grayish/PyVirtualEnv/py36-tensorfl